In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

In [2]:
from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob
import json
from tqdm.notebook import tqdm

# 1. labeling_json 가져와서 csv파일로 저장

## 1) damage

In [26]:
train_label_list = glob.glob("C:\\eun01\\car_repair\\training\\labeling_data\\damage\\*")

In [27]:
print(len(train_label_list))

403559


In [28]:
print(train_label_list[1])

C:\eun01\car_repair\training\labeling_data\damage\0000003_as-0036229.json


In [29]:
train_label_list[10000].split("\\")[-1][:7]

'0012517'

In [30]:
image_id=[]
category_id=[]
segmentation=[]
area=[]
bbox=[]
damage=[]
year=[]
color=[]
repair=[]
supercategory_name=[]

num=0  # 000001 과 같이 폴더명에 있는 이미지 파일 이름 가져오기
for file_name in tqdm(train_label_list):
    with open(file_name,'r') as j:
        contents=json.loads(j.read())

    for i in range(len(contents["annotations"])):
        image_id.append(train_label_list[num].split("\\")[-1][:7])
        category_id.append(contents["annotations"][i]["category_id"])
        segmentation.append(contents["annotations"][i]["segmentation"])
        area.append(contents["annotations"][i]["area"])
        bbox.append(contents["annotations"][i]["bbox"])
        damage.append(contents["annotations"][i]["damage"])
        year.append(contents["annotations"][i]["year"])
        color.append(contents["annotations"][i]["color"])
        repair.append(contents["annotations"][i]["repair"])
        supercategory_name.append(contents["categories"]["supercategory_name"])
    num+=1
        
car_repair1=pd.DataFrame({"image_id":image_id, "category_id":category_id, "segmentation":segmentation,
                         "area":area, "bbox":bbox, "damage":damage, "year":year, "color":color, "repair":repair,
                         "supercategory_name":supercategory_name})

  0%|          | 0/403559 [00:00<?, ?it/s]

In [33]:
car_repair1["image_id"] = car_repair1['image_id'].astype(str)
car_repair1.dtypes

image_id               object
category_id            object
segmentation           object
area                  float64
bbox                   object
damage                 object
year                    int64
color                  object
repair                 object
supercategory_name     object
dtype: object

In [34]:
car_repair1.tail()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
1237498,0506231,sc-202337,"[[[[341, 487], [312, 397], [271, 338], [298, 3...",9806.5,"[271, 317, 122, 175]",Scratched,2017,White,"[Front bumper:coating,exchange,sheet_metal]",CityCar
1237499,0506231,sc-202337,"[[[[401, 451], [407, 427], [415, 391], [408, 4...",142.0,"[401, 391, 14, 60]",Separated,2017,White,"[Front bumper:coating,exchange,sheet_metal]",CityCar
1237500,0506232,sc-202337,"[[[[646, 358], [661, 337], [713, 294], [736, 3...",20154.0,"[646, 294, 154, 213]",Scratched,2017,White,"[Front bumper:coating,exchange,sheet_metal]",CityCar
1237501,0506232,sc-202337,"[[[[186, 271], [246, 235], [362, 207], [452, 2...",2812.0,"[186, 207, 268, 73]",Scratched,2017,White,"[Front bumper:coating,exchange,sheet_metal]",CityCar
1237502,0506232,sc-202337,"[[[[592, 355], [705, 307], [799, 223], [800, 2...",3578.0,"[592, 223, 208, 154]",Separated,2017,White,"[Front bumper:coating,exchange,sheet_metal]",CityCar


### save

In [35]:
v1 = car_repair1[:600000]
v1.to_csv("car_repair1.csv", index=False)

In [36]:
v2 = car_repair1[600000:]
v2.to_csv("car_repair2.csv", index=False)

In [37]:
car_repair = pd.read_csv("car_repair1.csv")
car_repair.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,2,as-0036229,"[[[[438, 504], [440, 443], [436, 419], [439, 4...",14977.0,"[336, 159, 198, 345]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
1,3,as-0036229,"[[[[469, 497], [448, 479], [458, 437], [461, 4...",7186.5,"[378, 209, 154, 288]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
2,4,as-0036229,"[[[[73, 596], [69, 588], [67, 582], [65, 575],...",749.5,"[53, 543, 31, 53]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
3,4,as-0036229,"[[[[107, 371], [108, 362], [445, 361], [795, 3...",13872.5,"[107, 356, 688, 29]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
4,5,as-0036229,"[[[[77, 390], [87, 375], [452, 351], [560, 356...",22816.0,"[77, 351, 690, 52]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car


In [38]:
car_repair1 = pd.read_csv("car_repair2.csv")
car_repair1.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,246195,as-0073313,"[[[[496, 316], [508, 320], [510, 334], [480, 4...",1207.5,"[473, 316, 37, 91]",Scratched,2017,Gray tone,"['Rear bumper:coating,exchange']",Mid-size car
1,246195,as-0073313,"[[[[534, 386], [530, 408], [514, 403], [517, 3...",281.0,"[514, 383, 20, 25]",Scratched,2017,Gray tone,"['Rear bumper:coating,exchange']",Mid-size car
2,246195,as-0073313,"[[[[233, 261], [226, 269], [254, 277], [270, 2...",628.0,"[226, 258, 55, 19]",Scratched,2017,Gray tone,"['Rear bumper:coating,exchange']",Mid-size car
3,246195,as-0073313,"[[[[330, 283], [328, 331], [337, 326], [339, 2...",401.0,"[328, 283, 11, 48]",Breakage,2017,Gray tone,"['Rear bumper:coating,exchange']",Mid-size car
4,246195,as-0073313,"[[[[488, 222], [482, 234], [477, 275], [491, 2...",893.0,"[477, 222, 28, 54]",Breakage,2017,Gray tone,"['Rear bumper:coating,exchange']",Mid-size car


## 2) damage_part

In [5]:
train_label_list2 = glob.glob("C:\\eun01\\car_repair\\training\\labeling_data\\damage_part\\*")

In [6]:
print(len(train_label_list2))

109062


In [7]:
image_id=[]
category_id=[]
segmentation=[]
area=[]
bbox=[]
damage=[]
year=[]
color=[]
repair=[]
supercategory_name=[]

num=0
for file_name in tqdm(train_label_list2):
    with open(file_name,'r') as j:
        contents2=json.loads(j.read())

    for i in range(len(contents2["annotations"])):
        image_id.append(train_label_list2[num].split("\\")[-1][:7])
        category_id.append(contents2["annotations"][i]["category_id"])
        segmentation.append(contents2["annotations"][i]["segmentation"])
        area.append(contents2["annotations"][i]["area"])
        bbox.append(contents2["annotations"][i]["bbox"])
        damage.append(contents2["annotations"][i]["damage"])
        year.append(contents2["annotations"][i]["year"])
        color.append(contents2["annotations"][i]["color"])
        repair.append(contents2["annotations"][i]["repair"])
        supercategory_name.append(contents2["categories"]["supercategory_name"])
    num+=1
        
car_repair2=pd.DataFrame({"image_id":image_id, "category_id":category_id, "segmentation":segmentation,
                          "area":area, "bbox":bbox, "damage":damage, "year":year, "color":color, "repair":repair,
                          "supercategory_name":supercategory_name})

  0%|          | 0/109062 [00:00<?, ?it/s]

In [ ]:
car_repair2["image_id"] = car_repair2['image_id'].astype(str)
car_repair2["image_id"].dtypes

In [8]:
car_repair2.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,0000001,as-2866890,"[[[[234, 441], [231, 419], [255, 401], [281, 3...",865.5,"[231, 389, 56, 52]",Scratched,2019,Others,[Front fender(L):sheet_metal],Full-size car
1,0000001,as-2866890,"[[[[523, 33], [562, 28], [601, 71], [604, 91],...",102580.0,"[9, 28, 598, 425]",None,2019,Others,Front fender(L):sheet_metal,Full-size car
2,0000008,as-0036229,"[[[[206, 530], [222, 520], [256, 521], [273, 5...",3154.5,"[89, 505, 43, 19]",Separated,2020,Black,"[Rear bumper:coating,exchange]",Full-size car
3,0000008,as-0036229,"[[[[204, 517], [197, 533], [195, 521], [194, 5...",282.0,"[177, 503, 27, 30]",Separated,2020,Black,"[Rear bumper:coating,exchange]",Full-size car
4,0000008,as-0036229,"[[[[59, 356], [85, 367], [118, 372], [130, 418...",78361.5,"[59, 356, 665, 217]",None,2020,Black,"Rear bumper:coating,exchange",Full-size car


### save

In [10]:
car_repair2.to_csv("car_repair3.csv", index=False)

In [39]:
car_repair2 = pd.read_csv("car_repair3.csv")
car_repair2.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,1,as-2866890,"[[[[234, 441], [231, 419], [255, 401], [281, 3...",865.5,"[231, 389, 56, 52]",Scratched,2019,Others,['Front fender(L):sheet_metal'],Full-size car
1,1,as-2866890,"[[[[523, 33], [562, 28], [601, 71], [604, 91],...",102580.0,"[9, 28, 598, 425]",NaN,2019,Others,Front fender(L):sheet_metal,Full-size car
2,8,as-0036229,"[[[[206, 530], [222, 520], [256, 521], [273, 5...",3154.5,"[89, 505, 43, 19]",Separated,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
3,8,as-0036229,"[[[[204, 517], [197, 533], [195, 521], [194, 5...",282.0,"[177, 503, 27, 30]",Separated,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
4,8,as-0036229,"[[[[59, 356], [85, 367], [118, 372], [130, 418...",78361.5,"[59, 356, 665, 217]",NaN,2020,Black,"Rear bumper:coating,exchange",Full-size car


# 2. csv 파일 danage별로 저장

In [42]:
car_repair_all = pd.concat([car_repair, car_repair1, car_repair2], ignore_index=True)

In [21]:
car_repair_all.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,2,as-0036229,"[[[[438, 504], [440, 443], [436, 419], [439, 4...",14977.0,"[336, 159, 198, 345]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
1,3,as-0036229,"[[[[469, 497], [448, 479], [458, 437], [461, 4...",7186.5,"[378, 209, 154, 288]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
2,4,as-0036229,"[[[[73, 596], [69, 588], [67, 582], [65, 575],...",749.5,"[53, 543, 31, 53]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
3,4,as-0036229,"[[[[107, 371], [108, 362], [445, 361], [795, 3...",13872.5,"[107, 356, 688, 29]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
4,5,as-0036229,"[[[[77, 390], [87, 375], [452, 351], [560, 356...",22816.0,"[77, 351, 690, 52]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car


In [43]:
car_repair_all = car_repair_all.sort_values("image_id")
car_repair_all.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
1237503,1,as-2866890,"[[[[234, 441], [231, 419], [255, 401], [281, 3...",865.5,"[231, 389, 56, 52]",Scratched,2019,Others,['Front fender(L):sheet_metal'],Full-size car
1237504,1,as-2866890,"[[[[523, 33], [562, 28], [601, 71], [604, 91],...",102580.0,"[9, 28, 598, 425]",NaN,2019,Others,Front fender(L):sheet_metal,Full-size car
0,2,as-0036229,"[[[[438, 504], [440, 443], [436, 419], [439, 4...",14977.0,"[336, 159, 198, 345]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
1,3,as-0036229,"[[[[469, 497], [448, 479], [458, 437], [461, 4...",7186.5,"[378, 209, 154, 288]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
3,4,as-0036229,"[[[[107, 371], [108, 362], [445, 361], [795, 3...",13872.5,"[107, 356, 688, 29]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car


In [44]:
len(car_repair_all)

1800216

In [45]:
car_repair_all["damage"].unique()

array(['Scratched', nan, 'Breakage', 'Separated', 'Crushed'], dtype=object)

In [52]:
Scratched = car_repair_all[car_repair_all["damage"]=="Scratched"]
Breakage = car_repair_all[car_repair_all["damage"]=="Breakage"]
Separated = car_repair_all[car_repair_all["damage"]=="Separated"]
Crushed = car_repair_all[car_repair_all["damage"]=="Crushed"]
Null = car_repair_all[car_repair_all["damage"].isnull()]

In [53]:
Scratched.to_csv("Scratched.csv", index=False)
Breakage.to_csv("Breakage.csv", index=False)
Separated.to_csv("Separated.csv", index=False)
Crushed.to_csv("Crushed.csv", index=False)
Null.to_csv("Null.cav", index=True)